In [1]:
import warnings

import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.utils.validation import check_is_fitted
from sklearn.pipeline import Pipeline, make_pipeline

warnings.simplefilter(action="ignore", category=FutureWarning)

In this lesson, we're going to build on the work we did in the previous lesson. We're going to create a more complex wrangle function, use it to clean more data, and build a model that considers more features when predicting apartment price

# Prepare Data
## Import

In [2]:
def wrangle(filepath):
    # Read CSV file
    df = pd.read_csv(filepath)
    #subset to "capital federal"
    
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    #subset to "Apartment"
    mask_apt = df["property_type"] == "apartment"
    #subset to proce_usd
    mask_price = df["price_aprox_usd"] < 400_000
    df = df[mask_ba & mask_apt & mask_price]
    
    #subset to area
    low , high = df["surface_covered_in_m2"].quantile([0.1, 0.9])
    mask_area = df["surface_covered_in_m2"].between(low,high)
    df = df[mask_area]


#     split "lat-lon"
    df[["lat","lon"]] = df["lat-lon"].str.split(",",expand = True).astype(float)
    df.drop(columns="lat-lon",inplace = True)
    df.drop(columns="Unnamed: 0",inplace = True)

    return df

<b> Task 2.2.1: Use your wrangle function to create a DataFrame frame1 from the CSV file data/buenos-aires-real-estate-1.csv.</b>

In [3]:
frame1 = wrangle(r"C:\Users\MEER\Desktop\applied datascience course\project 2 house price prediction\buenos-aires-real-estate-1.csv")
print("frame1 shape:", frame1.shape)
print(frame1.info())
frame1.head()

frame1 shape: (1343, 17)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1343 entries, 4 to 8604
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   operation                   1343 non-null   object 
 1   property_type               1343 non-null   object 
 2   place_with_parent_names     1343 non-null   object 
 3   price                       1343 non-null   float64
 4   currency                    1343 non-null   object 
 5   price_aprox_local_currency  1343 non-null   float64
 6   price_aprox_usd             1343 non-null   float64
 7   surface_total_in_m2         965 non-null    float64
 8   surface_covered_in_m2       1343 non-null   float64
 9   price_usd_per_m2            927 non-null    float64
 10  price_per_m2                1343 non-null   float64
 11  floor                       379 non-null    float64
 12  rooms                       1078 non-null   float64
 13  expenses

,operation,property_type,place_with_parent_names,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,lat,lon
4,sell,apartment,|Argentina|Capital Federal|Chacarita|,129000.0,USD,1955949.6,129000.0,76.0,70.0,1697.368421,1842.857143,NaN,NaN,NaN,http://chacarita.properati.com.ar/10qlv_venta_...,-34.584651,-58.454693
9,sell,apartment,|Argentina|Capital Federal|Villa Luro|,87000.0,USD,1319128.8,87000.0,48.0,42.0,1812.500000,2071.428571,NaN,NaN,NaN,http://villa-luro.properati.com.ar/12m82_venta...,-34.638979,-58.500115
29,sell,apartment,|Argentina|Capital Federal|Caballito|,118000.0,USD,1789163.2,118000.0,NaN,54.0,NaN,2185.185185,NaN,2.0,NaN,http://caballito.properati.com.ar/11wqh_venta_...,-34.615847,-58.459957
40,sell,apartment,|Argentina|Capital Federal|Constituci�n|,57000.0,USD,864256.8,57000.0,42.0,42.0,1357.142857,1357.142857,5.0,2.0,364.0,http://constitucion.properati.com.ar/k2f0_vent...,-34.625222,-58.382382
41,sell,apartment,|Argentina|Capital Federal|Once|,90000.0,USD,1364616.0,90000.0,57.0,50.0,1578.947368,1800.000000,NaN,3.0,450.0,http://once.properati.com.ar/suwa_venta_depart...,-34.610610,-58.412511


For our model, we're going to consider apartment location, specifically, latitude and longitude. Looking at the output from frame1.info(), we can see that the location information is in a single column where the data type is object (pandas term for str in this case). In order to build our model, we need latitude and longitude to each be in their own column where the data type is float.

<b> Task 2.2.2: Add to the wrangle function below so that, in the DataFrame it returns, the "lat-lon" column is replaced by separate "lat" and "lon" columns. Don't forget to also drop the "lat-lon" column. Be sure to rerun all the cells above before you continue.</b>

In [4]:
frame1["lat-lon"].str.split(",",expand = True).astype(float).head()

KeyError: 'lat-lon'

In [ ]:
# Check your work
assert (
    frame1.shape[0] == 1343
), f"`frame1` should have 1343 rows, not {frame1.shape[0]}."
assert frame1.shape[1] == 17, f"`frame1` should have 17 columns, not {frame1.shape[1]}."

<b> Task 2.2.3: Use you revised wrangle function create a DataFrames frame2 from the file data/buenos-aires-real-estate-2.csv.</b>

In [ ]:
frame2 = wrangle(r"C:\Users\MEER\Desktop\applied datascience course\project 2 house price prediction\buenos-aires-real-estate-2.csv")

In [ ]:
# Check your work
assert (
    frame2.shape[0] == 1315
), f"`frame1` should have 1315 rows, not {frame2.shape[0]}."
assert frame2.shape[1] == 17, f"`frame1` should have 17 columns, not {frame2.shape[1]}."

As you can see, using a function is much quicker than cleaning each file individually like we did in the last project. Let's combine our DataFrames so we can use then to train our model.

<b> Task 2.2.4: Use pd.concat to concatenate frame1 and frame2 into a new DataFrame df. Make sure you set the ignore_index argument to True.</b>

In [ ]:
df = pd.concat([frame1,frame2] , ignore_index=True)
print(df.info())
df.head()

In [ ]:
# Check your work
assert df.shape == (2658, 17), f"`df` is the wrong size: {df.shape}"

# Explore
In the last lesson, we built a simple linear model that predicted apartment price based on one feature, "surface_covered_in_m2". In this lesson, we're building a multiple linear regression model that predicts price based on two features, "lon" and "lat". This means that our data visualizations now have to communicate three pieces of information: Longitude, latitude, and price. How can we represent these three attributes on a two-dimensional screen?

One option is to incorporate color into our scatter plot. For example, in the Mapbox scatter plot below, the location of each point represents latitude and longitude, and color represents price.

<b> Task 2.2.5: Complete the code below to create a Mapbox scatter plot that shows the location of the apartments in df.</b>

In [5]:
fig = px.scatter_mapbox(
    df,  # Our DataFrame
    lat="lat",
    lon="lon",
    width=600,  # Width of map
    height=600,  # Height of map
    color="price_aprox_usd",
    hover_data=["price_aprox_usd"],  # Display price when hovering mouse over house
)

fig.update_layout(mapbox_style="open-street-map")

fig.show()

NameError: name 'df' is not defined

Another option is to add a third dimension to our scatter plot. We can plot longitude on the x-axis and latitude on the y-axis (like we do in the map above), and then add a z-axis with price.

<b> Task 2.2.6: Complete the code below to create a 3D scatter plot, with "lon" on the x-axis, "lat" on the y-axis, and "price_aprox_usd" on the z-axis.</b>

In [6]:
# Create 3D scatter plot
fig = px.scatter_3d(
    df,
    x="lat",
    y="lon",
    z="price_aprox_usd",
    labels={"lon": "longitude", "lat": "latitude", "price_aprox_usd": "price"},
    width=600,
    height=500,
)

# Refine formatting
fig.update_traces(
    marker={"size": 4, "line": {"width": 2, "color": "DarkSlateGrey"}},
    selector={"mode": "markers"},
)

# Display figure
fig.show()

NameError: name 'df' is not defined

# Tip:
3D visualizations are often harder for someone to interpret than 2D visualizations. We're using one here because it will help us visualize our model once it's built, but as a rule, it's better to stick with 2D when your communicating with an audience.

# Split
Even though we're building a different model, the steps we follow will be the same. Let's separate our features (latitude and longitude) from our target (price).

<b> Task 2.2.7: Create the feature matrix named X_train. It should contain two features: ["lon", "lat"].</b>

In [7]:
features = ["lon", "lat"]
X_train = df[features]

NameError: name 'df' is not defined

<b> Task 2.2.8: Create the target vector named y_train, which you'll use to train your model. Your target should be "price_aprox_usd". Remember that, in most cases, your target vector should be one-dimensional.</b>

In [8]:
target = "price_aprox_usd"
y_train = df[target]
y_train.shape

NameError: name 'df' is not defined

# Build Model
### Baseline
Again, we need to set a baseline so we can evaluate our model's performance. You'll notice that the value of y_mean is not exactly the same as it was in the previous lesson. That's because we've added more observations to our training data.

<b> Task 2.2.9: Calculate the mean of your target vector y_train and assign it to the variable y_mean.</b>

In [9]:
y_mean = y_train.mean()
y_pred_baseline = [y_mean] * len(y_train)
y_pred_baseline[:5]

NameError: name 'y_train' is not defined

<b> Task 2.2.11: Calculate the baseline mean absolute error for your predictions in y_pred_baseline as compared to the true targets in y_train.</b>

In [10]:
mae_baseline = mean_absolute_error(y_train , y_pred_baseline)

print("Mean apt price", round(y_mean, 2))
print("Baseline MAE:", round(mae_baseline, 2))

NameError: name 'y_train' is not defined

# Iterate
Take a moment to scroll up to the output for df.info() and look at the values in the "Non-Null Count" column. Because of the math it uses, a linear regression model can't handle observations where there are missing values. Do you see any columns where this will be a problem?

In the last project, we simply dropped rows that contained NaN values, but this isn't ideal. Models generally perform better when they have more data to train with, so every row is precious. Instead, we can fill in these missing values using information we get from the whole column — a process called imputation. There are many different strategies for imputing missing values, and one of the most common is filling in the missing values with the mean of the column.

In addition to predictors like LinearRegression, scikit-learn also has transformers that help us deal with issues like missing values. Let's see how one works, and then we'll add it to our model.

<b> Task 2.2.12: Instantiate a SimpleImputer named imputer.</b>

In [11]:
imputer = SimpleImputer()

In [12]:
# Check your work
assert isinstance(imputer, SimpleImputer)

Just like a predictor, a transformer has a fit method. In the case of our SimpleImputer, this is the step where it calculates the mean values for each numerical column.

<b> Task 2.2.13: Fit your transformer imputer to the feature matrix X.</b>

In [13]:
imputer.fit(X_train)

NameError: name 'X_train' is not defined

In [14]:
# Check your work
check_is_fitted(imputer)

NotFittedError: This SimpleImputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

Here's where transformers diverge from predictors. Instead of using a method like predict, we use the transform method. This is the step where the transformer fills in the missing values with the means it's calculated.

<b> Task 2.2.14: Use your imputer to transform the feature matrix X_train. Assign the transformed data to the variable XT_train.</b>

In [15]:
XT_train = imputer.transform(X_train)
pd.DataFrame(XT_train, columns=X_train.columns).info()

NameError: name 'X_train' is not defined

In [16]:
# Check your work
assert XT_train.shape == (2658, 2), f"`XT_train` is the wrong shape: {XT_train.shape}"
assert (
    np.isnan(XT_train).sum() == 0
), "Your feature matrix still has `NaN` values. Did you forget to transform is using `imputer`?"

NameError: name 'XT_train' is not defined

Okay! Our data is free of missing values, and we have a good sense for how predictors work in scikit-learn. However, the truth is you'll rarely do data transformations this way. Why? A model may require multiple transformers, and doing all those transformations one-by-one is slow and likely to lead to errors. 🤦‍♀️ Instead, we can combine our transformer and predictor into a single object called a pipeline. WQU WorldQuant University Applied Data Science Lab QQQQ

<b> Task 2.2.15: Create a pipeline named model that contains a SimpleImputer transformer followed by a LinearRegression predictor.</b>

In [17]:
model = make_pipeline(
    SimpleImputer(),
    LinearRegression()
)

In [18]:
assert isinstance(model, Pipeline), "Did you instantiate your model?"

With our pipeline assembled, we use the fit method, which will train the transformer, transform the data, then pass the transformed data to the predictor for training, all in one step. Much easier!

<b> Task 2.2.16: Fit your model to the data, X_train and y_train.</b>

In [19]:
model.fit(X_train,y_train)

NameError: name 'X_train' is not defined

In [20]:
# Check your work
check_is_fitted(model["linearregression"])

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

# Evaluate
As always, we'll start by evaluating our model's performance on the training data.

<b> Task 2.2.17: Using your model's predict method, create a list of predictions for the observations in your feature matrix X_train. Name this list y_pred_training.</b>

In [21]:
y_pred_training = model.predict(X_train)

NameError: name 'X_train' is not defined

In [22]:
# Check your work
assert y_pred_training.shape == (2658,)

NameError: name 'y_pred_training' is not defined

<b> Task 2.2.18: Calculate the training mean absolute error for your predictions in y_pred_training as compared to the true targets in y_train</b>

In [23]:
mae_training = mean_absolute_error(y_train,y_pred_training)
print("Training MAE:", round(mae_training, 2))

NameError: name 'y_train' is not defined

It looks like our model performs a little better than the baseline. This suggests that latitude and longitude aren't as strong predictors of price as size is.

Now let's check our test performance. Remember, once we test our model, there's no more iteration allowed.

# Communicate Results
Let's take a look at the equation our model has come up with for predicting price based on latitude and longitude. We'll need to expand on our formula to account for both features.

Equation: y = beta 0 + beta 1 * x

<b> Task 2.2.20: Extract the intercept and coefficients for your model.</b>

In [24]:
intercept = model.named_steps["linearregression"].intercept_
coefficients = model.named_steps["linearregression"].coef_

AttributeError: 'LinearRegression' object has no attribute 'intercept_'

<b> Task 2.2.21: Complete the code below and run the cell to print the equation that your model has determined for predicting apartment price based on latitude and longitude.</b>

In [25]:
print(
    
    f"price = {intercept} + ({coefficients} * longitude) + ({coefficients} * latitude)"
)

NameError: name 'intercept' is not defined

What does this equation tell us? As you move north and west, the predicted apartment price increases.

At the start of the notebook, you thought about how we would represent our linear model in a 3D plot. If you guessed that we would use a plane, you're right!

<b> Task 2.2.22: Complete the code below to create a 3D scatter plot, with "lon" on the x-axis, "lat" on the y-axis, and "price_aprox_usd" on the z-axis.</b>

In [26]:
# Create 3D scatter plot
fig = px.scatter_3d(
    df,
    x="lon",
    y="lat",
    z="price_aprox_usd",
    labels={"lon": "longitude", "lat": "latitude", "price_aprox_usd": "price"},
    width=600,
    height=500,
)

# Create x and y coordinates for model representation
x_plane = np.linspace(df["lon"].min(), df["lon"].max(), 10)
y_plane = np.linspace(df["lat"].min(), df["lat"].max(), 10)
xx, yy = np.meshgrid(x_plane, y_plane)

# Use model to predict z coordinates
z_plane = model.predict(pd.DataFrame({"lon": x_plane, "lat": y_plane}))
zz = np.tile(z_plane, (10, 1))

# Add plane to figure
fig.add_trace(go.Surface(x=xx, y=yy, z=zz))
# Refine formatting
fig.update_traces(
    marker={"size": 4, "line": {"width": 2, "color": "DarkSlateGrey"}},
    selector={"mode": "markers"},
)

# Display figure
fig.show()

NameError: name 'df' is not defined